Pyspark ML

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('spark').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/11 23:03:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_pyspark = spark.read.csv('test6.csv', header=True, inferSchema=True)

In [5]:
df_pyspark.show()

+--------+---+----------+------+
|    Name|age|Experience|Salary|
+--------+---+----------+------+
|   Krish| 31|        10| 30000|
| Sunhand| 30|         8| 50000|
|   Sunny| 29|         4| 20000|
|    Paul| 24|         3| 15000|
|Shuhaham| 21|         1| 18000|
+--------+---+----------+------+



In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
# VectorAssembler = group age and experience columns
va = VectorAssembler(inputCols=["age","Experience"], outputCol="Indepennt Feature")

In [9]:
output = va.transform(df_pyspark)

In [10]:
output.show()

+--------+---+----------+------+-----------------+
|    Name|age|Experience|Salary|Indepennt Feature|
+--------+---+----------+------+-----------------+
|   Krish| 31|        10| 30000|      [31.0,10.0]|
| Sunhand| 30|         8| 50000|       [30.0,8.0]|
|   Sunny| 29|         4| 20000|       [29.0,4.0]|
|    Paul| 24|         3| 15000|       [24.0,3.0]|
|Shuhaham| 21|         1| 18000|       [21.0,1.0]|
+--------+---+----------+------+-----------------+



In [12]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Indepennt Feature']

In [15]:
f_data = output.select('Indepennt Feature','Salary')

In [16]:
f_data.show()

+-----------------+------+
|Indepennt Feature|Salary|
+-----------------+------+
|      [31.0,10.0]| 30000|
|       [30.0,8.0]| 50000|
|       [29.0,4.0]| 20000|
|       [24.0,3.0]| 15000|
|       [21.0,1.0]| 18000|
+-----------------+------+



In [17]:
from pyspark.ml.regression import LinearRegression

In [21]:
# randomSplit = training data will have 75% of data, test data will have 25%
# labelCol = output
train_data, test_data = f_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol="Indepennt Feature", labelCol="Salary")
regressor = regressor.fit(train_data)

24/08/11 23:11:44 WARN Instrumentation: [9dda2037] regParam is zero, which might cause numerical instability and overfitting.
24/08/11 23:11:45 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/11 23:11:45 WARN Instrumentation: [9dda2037] The standard deviation of the label is zero, so the coefficients will be zeros and the intercept will be the mean of the label; as a result, training is not needed.


In [23]:
regressor.coefficients

DenseVector([0.0, 0.0])

In [24]:
regressor.intercept

20000.0

In [26]:
pred_results = regressor.evaluate(test_data)

In [28]:
pred_results.predictions.show()

+-----------------+------+----------+
|Indepennt Feature|Salary|prediction|
+-----------------+------+----------+
|       [21.0,1.0]| 18000|   20000.0|
|       [24.0,3.0]| 15000|   20000.0|
|       [30.0,8.0]| 50000|   20000.0|
|      [31.0,10.0]| 30000|   20000.0|
+-----------------+------+----------+

